In [52]:
import pandas as pd
import numpy as np
import data_preprocess as dp
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt

In [113]:
data = pd.read_csv('./data/D2.csv')

In [114]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   race                      20000 non-null  object
 1   gender                    20000 non-null  object
 2   age                       20000 non-null  object
 3   admission_type_id         20000 non-null  int64 
 4   discharge_disposition_id  20000 non-null  int64 
 5   admission_source_id       20000 non-null  int64 
 6   time_in_hospital          20000 non-null  int64 
 7   medical_specialty         20000 non-null  object
 8   num_lab_procedures        20000 non-null  int64 
 9   num_procedures            20000 non-null  int64 
 10  num_medications           20000 non-null  int64 
 11  number_outpatient         20000 non-null  int64 
 12  number_emergency          20000 non-null  int64 
 13  number_inpatient          20000 non-null  int64 
 14  number_diagnoses      

In [55]:
df = dp.data_prep(data)

In [56]:
# drop the rows with where discharge_disposition is deceased
df = df[df.discharge_disposition != 'Deceased']

In [57]:
# Step 1: Separating the features and target variable
X = df.drop('readmitted', axis=1)  # Features
y = df['readmitted']  # Target

In [58]:
# Identifying categorical columns that need encoding
categorical_columns = ['race', 'medical_specialty', 'admission_type', 'discharge_disposition', 'admission_source']
numerical_columns = ['gender', 'age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications',
                     'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses',
                     'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
                     'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'insulin', 'change', 'diabetesMed']

In [59]:
label_encoder = LabelEncoder()
X['age'] = label_encoder.fit_transform(X['age'])

In [60]:
X = pd.get_dummies(X, columns=categorical_columns, dtype=int)

In [61]:
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])


In [62]:
from sklearn.model_selection import train_test_split

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [64]:
# Import the model
from sklearn.linear_model import LogisticRegression

# Instantiate the default model
model = LogisticRegression(random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [65]:
from sklearn.metrics import classification_report, roc_curve, roc_auc_score

# training and test accuracy
print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))

# classification report on test data
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

Train accuracy: 0.6307588075880759
Test accuracy: 0.6302845528455284
              precision    recall  f1-score   support

           0       0.62      0.77      0.69      2588
           1       0.65      0.48      0.55      2332

    accuracy                           0.63      4920
   macro avg       0.63      0.62      0.62      4920
weighted avg       0.63      0.63      0.62      4920



In [66]:
# Getting the coefficients for all features
coefficients = model.coef_[0]

In [67]:
coefficients

array([ 0.00824324,  0.07173757, -0.02837418,  0.02883632, -0.03268487,
        0.08975476,  0.1266453 ,  0.3086645 ,  0.55378323,  0.07900704,
       -0.06029132, -0.00939959, -0.02359888,  0.12041952, -0.01301767,
        0.        , -0.00346593,  0.02808425,  0.        ,  0.03088719,
        0.02377524,  0.10901493,  0.00524116, -0.11870695,  0.20453181,
       -0.36593355,  0.07177846,  0.0651096 ,  0.07460423, -0.25122657,
        0.10737214, -0.19894847, -0.22017638, -0.0777035 ,  0.0947908 ,
       -0.08498437, -0.06211351, -0.05599119,  0.14306664,  0.12551335,
       -0.38607914, -0.08558991])

In [68]:
# grab feature importances from the model and feature name from the original X
coef = model.coef_[0]
feature_names = X.columns

# sort them out in descending order
indices = np.argsort(np.absolute(coef))
indices = np.flip(indices, axis=0)

# limit to 20 features, you can leave this out to print out everything
indices = indices[:20]

for i in indices:
    print(f"{feature_names[i]} -> {coef[i]:.3f}")

number_inpatient -> 0.554
admission_source_Transfer -> -0.386
race_Hispanic -> -0.366
number_emergency -> 0.309
medical_specialty_Internal Medicine -> -0.251
admission_type_Elective -> -0.220
race_Caucasian -> 0.205
medical_specialty_Surgical -> -0.199
admission_source_Emergency -> 0.143
number_outpatient -> 0.127
admission_source_Referral -> 0.126
chlorpropamide -> 0.120
race_Asian -> -0.119
diabetesMed -> 0.109
medical_specialty_Other -> 0.107
admission_type_Unknown -> 0.095
num_medications -> 0.090
admission_source_Unknown -> -0.086
discharge_disposition_AdditonalCare -> -0.085
number_diagnoses -> 0.079


In [69]:
# perform cross validation with gridsearchCV
from sklearn.model_selection import GridSearchCV


In [98]:
# defining the hyperparameters
params = {
    'C': [pow(10, x) for x in range(-6, 4)],

}

In [99]:
# Instantiating the gridsearch
grid_search = GridSearchCV(param_grid=params, estimator=LogisticRegression(random_state=1), return_train_score=True,
                           cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=5, estimator=LogisticRegression(random_state=1), n_jobs=-1,
             param_grid={'C': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10,
                               100, 1000]},
             return_train_score=True, verbose=1)

In [104]:
# best hyperparameters  
print(f"Best parameters found: {grid_search.best_params_}")

Best parameters found: {'C': 0.01}


In [105]:
best_model = grid_search.best_estimator_

In [106]:
best_model

LogisticRegression(C=0.01, random_state=1)

In [109]:
coefs = best_model.coef_[0]
importance = np.abs(coefs)
feature_names = X.columns

# sort them out in descending order
indices = np.argsort(importance)
indices = np.flip(indices, axis=0)

# limit to 5 features
indices = indices[:5]

top_5_features = feature_names[indices]
top_5_importance = importance[indices]

for i in range(5):
    print(f"{top_5_features[i]} -> {top_5_importance[i]:.3f}")


number_inpatient -> 0.527
number_emergency -> 0.279
medical_specialty_Internal Medicine -> 0.181
admission_source_Transfer -> 0.178
race_Caucasian -> 0.169


In [112]:
# Training and test accuracy
from sklearn.metrics import accuracy_score

# Best model from GridSearchCV
best_model = grid_search.best_estimator_

# Predict on the training and test sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate training and test accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print the accuracy scores
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Training Accuracy: 0.6314
Test Accuracy: 0.6307


In [101]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_test_pred)
print(conf_matrix)

[[2007  581]
 [1236 1096]]


In [102]:
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best accuracy score: {grid_search.best_score_}")

Best parameters found: {'C': 0.01}
Best accuracy score: 0.6276422764227642


In [82]:
results = pd.DataFrame(grid_search.cv_results_)

In [83]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.044611,0.004870,0.003922,0.001174,0.000001,liblinear,"{'C': 1e-06, 'solver': 'liblinear'}",0.599593,0.596206,0.603320,...,0.602507,0.004240,24,0.602473,0.605776,0.602219,0.604929,0.604844,0.604048,0.001430
1,0.037755,0.007147,0.004620,0.001399,0.000001,lbfgs,"{'C': 1e-06, 'solver': 'lbfgs'}",0.526084,0.526423,0.526423,...,0.526355,0.000136,29,0.526423,0.526338,0.526338,0.526338,0.526338,0.526355,0.000034
2,0.173684,0.022172,0.006778,0.002366,0.000001,saga,"{'C': 1e-06, 'solver': 'saga'}",0.526084,0.526423,0.526423,...,0.526355,0.000136,29,0.526423,0.526338,0.526338,0.526338,0.526338,0.526355,0.000034
3,0.057225,0.003705,0.006081,0.001104,0.000010,liblinear,"{'C': 1e-05, 'solver': 'liblinear'}",0.602642,0.595867,0.605691,...,0.603726,0.004491,23,0.605014,0.608147,0.603997,0.605945,0.607131,0.606047,0.001474
4,0.034176,0.003628,0.004178,0.000194,0.000010,lbfgs,"{'C': 1e-05, 'solver': 'lbfgs'}",0.529133,0.530149,0.529810,...,0.530556,0.001121,27,0.531165,0.530911,0.530742,0.529810,0.530234,0.530572,0.000488
5,0.177792,0.002785,0.005425,0.000593,0.000010,saga,"{'C': 1e-05, 'solver': 'saga'}",0.529133,0.530149,0.529810,...,0.530556,0.001121,27,0.531165,0.530911,0.530742,0.529810,0.530234,0.530572,0.000488
6,0.082941,0.007988,0.005238,0.001611,0.000100,liblinear,"{'C': 0.0001, 'solver': 'liblinear'}",0.610772,0.606030,0.610095,...,0.611043,0.003471,22,0.612890,0.613906,0.611704,0.613482,0.614329,0.613262,0.000913
7,0.045189,0.001812,0.004016,0.001063,0.000100,lbfgs,"{'C': 0.0001, 'solver': 'lbfgs'}",0.602304,0.596206,0.601626,...,0.600949,0.002535,25,0.602473,0.603404,0.601203,0.603574,0.603404,0.602812,0.000893
8,0.218159,0.012172,0.004827,0.000494,0.000100,saga,"{'C': 0.0001, 'solver': 'saga'}",0.601965,0.596206,0.601965,...,0.600949,0.002462,25,0.602388,0.603828,0.601287,0.603404,0.603320,0.602846,0.000910
9,0.169953,0.029866,0.004607,0.000383,0.001000,liblinear,"{'C': 0.001, 'solver': 'liblinear'}",0.621274,0.620257,0.620935,...,0.622561,0.005085,19,0.625169,0.626524,0.625085,0.626524,0.627371,0.626135,0.000879


In [85]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.77      0.69      2588
           1       0.65      0.48      0.55      2332

    accuracy                           0.63      4920
   macro avg       0.63      0.62      0.62      4920
weighted avg       0.63      0.63      0.62      4920



In [86]:
mean_train_scores = results['mean_train_score']
mean_test_scores = results['mean_test_score']


In [ ]:
# Plot the results for max_depth
plt.figure(figsize=(10, 6))
plt.plot(mean_train_scores.index, mean_train_scores, label='Mean Train Score', marker='o', linestyle='--', color='b')
plt.plot(mean_test_scores.index, mean_test_scores, label='Mean Test Score', marker='x', linestyle='-', color='r')

# Add labels and title
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Train vs Test Accuracy for Max Depth (Averaged Across 10 CV Splits)')
plt.xticks(range(0, len(results)), [pow(10, x) for x in range(-6, 4)])
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
grid_search.best_params_

In [ ]:
# Using the best hyperparameters
best_model = LogisticRegression(C=0.01, random_state=0)
best_model.fit(X_train, y_train)

In [ ]:
# training and test accuracy
print("Train accuracy:", best_model.score(X_train, y_train))
print("Test accuracy:", best_model.score(X_test, y_test))


In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
rfe = RFECV(estimator=LogisticRegression(C=0.01, random_state=0), cv=10)
rfe.fit(X_train, y_train)

In [ ]:
print(f"Original feature set: {X_train.shape[1]}")
print(f"Optimal number of features : {rfe.n_features_}")

In [ ]:
X_train_sel = rfe.transform(X_train)
X_test_sel = rfe.transform(X_test)

In [ ]:
params

In [ ]:
rfe_grid_search = GridSearchCV(param_grid=params, estimator=LogisticRegression(random_state=0), return_train_score=True,
                               cv=10, n_jobs=-1, verbose=1)

In [ ]:
rfe_grid_search.fit(X_train_sel, y_train)

In [ ]:
print(f"Best parameters found: {rfe_grid_search.best_params_}")
print(f"Best accuracy score: {rfe_grid_search.best_score_}")
print(f"Train score: {rfe_grid_search.score(X_train_sel, y_train)}")
print(f"Test score: {rfe_grid_search.score(X_test_sel, y_test)}")



In [ ]:
y_pred = rfe_grid_search.predict(X_test_sel)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
coefs = model.coef_[0]
selected_coefs = coefs[rfe.support_]
feature_names = X.columns[rfe.support_]

# sort them out in descending order
indices = np.argsort(np.absolute(selected_coefs))
indices = np.flip(indices, axis=0)

for i in indices:
    print(f"{feature_names[i]} -> {selected_coefs[i]:.3f}")

In [ ]:
# Predicted probabilities for both models (for class 1)
y_proba_model = model.predict_proba(X_test)[:, 1]
y_proba_rfe = rfe_grid_search.predict_proba(X_test_sel)[:, 1]

# Compute ROC curve and AUC for both models
fpr_rfe, tpr_rfe, _ = roc_curve(y_test, y_proba_rfe)
roc_auc_rfe = roc_auc_score(y_test, y_proba_rfe)

fpr_model, tpr_model, _ = roc_curve(y_test, y_proba_model)
roc_auc_model = roc_auc_score(y_test, y_proba_model)

# Create subplots for the two models
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot ROC for best_model
ax[0].plot(fpr_rfe, tpr_rfe, color='orange', label=f'ROC Curve (AUC = {roc_auc_rfe:.2f})')
ax[0].plot([0, 1], [0, 1], color='blue', linestyle='--')
ax[0].set_xlabel('False Positive Rate (FPR)')
ax[0].set_ylabel('True Positive Rate (TPR)')
ax[0].set_title('Best Model ROC Curve')
ax[0].legend(loc='lower right')
ax[0].grid(True)

# Plot ROC for model
ax[1].plot(fpr_model, tpr_model, color='green', label=f'ROC Curve (AUC = {roc_auc_model:.2f})')
ax[1].plot([0, 1], [0, 1], color='blue', linestyle='--')
ax[1].set_xlabel('False Positive Rate (FPR)')
ax[1].set_ylabel('True Positive Rate (TPR)')
ax[1].set_title('Base Model ROC Curve')
ax[1].legend(loc='lower right')
ax[1].grid(True)

# Show the plots
plt.tight_layout()
plt.show()